# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [63]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,clyde river,70.4692,-68.5914,-31.87,75,0,1.54,CA,1679126442
1,1,karratha,-20.7377,116.8463,35.98,36,2,5.45,AU,1679126319
2,2,klaksvik,62.2266,-6.5890,2.51,59,100,9.34,FO,1679126443
3,3,hithadhoo,-0.6000,73.0833,27.85,70,100,0.98,MV,1679126443
4,4,avarua,-21.2078,-159.7750,26.03,89,100,2.57,CK,1679126276


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 450,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [65]:
# Narrow down cities that fit criteria and drop any results with null values
nice_weathre_city_data_df = city_data_df[(city_data_df["Max Temp"]>=21)
                                         & (city_data_df["Max Temp"]<30)
                                         & (city_data_df["Wind Speed"]<4.5)
                                         & (city_data_df["Cloudiness"]<10)
                                         & (city_data_df["Humidity"]<95)]

# Drop any rows with null values
nice_weathre_city_data_df = nice_weathre_city_data_df.dropna()

# Display sample data
nice_weathre_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,kapaa,22.0752,-159.3190,23.18,83,0,1.54,US,1679126074
10,10,lujan,-34.5703,-59.1050,22.79,92,0,0.89,AR,1679126403
17,17,arraial do cabo,-22.9661,-42.0278,23.63,85,0,1.12,BR,1679126086
50,50,victoria,22.2855,114.1577,26.65,78,6,4.02,HK,1679126220
69,69,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1679126073


### Step 3: Create a new DataFrame called `hotel_df`.

In [66]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = nice_weathre_city_data_df[{"City", "Country", "Lat", "Lng", "Humidity"}].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

/var/folders/n3/0fjc3g9d04gb6vgxphygsqj80000gn/T/ipykernel_93473/2350268650.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = nice_weathre_city_data_df[{"City", "Country", "Lat", "Lng", "Humidity"}].copy()


,Country,Humidity,Lat,City,Lng,Hotel Name
7,US,83,22.0752,kapaa,-159.3190,
10,AR,92,-34.5703,lujan,-59.1050,
17,BR,85,-22.9661,arraial do cabo,-42.0278,
50,HK,78,22.2855,victoria,114.1577,
69,US,83,19.7297,hilo,-155.0900,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

# Set the parameters for the type of search
limit = 1

# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
lujan - nearest hotel: Bella vita
arraial do cabo - nearest hotel: No hotel found
victoria - nearest hotel: 四季酒店 Four Seasons Hotel
hilo - nearest hotel: Dolphin Bay Hotel
acajutla - nearest hotel: Hotel Lara
gold coast - nearest hotel: The Sands Apartments
agadez - nearest hotel: No hotel found
aswan - nearest hotel: Yaseen hotel
maragogi - nearest hotel: Pousada Mariluz
mudgee - nearest hotel: Oriental Hotel
acapulco - nearest hotel: Hotel del Valle
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
bam - nearest hotel: هتل جهانگردی بم
iranshahr - nearest hotel: هتل قصِر
tuy hoa - nearest hotel: Vinh Tuan Hotel
goure - nearest hotel: No hotel found
pandamatenga - nearest hotel: No hotel found
sao jose da coroa grande - nearest hotel: Hotel do Sol
maturin - nearest hotel: Hotel Rossi
bafoulabe - nearest hotel: No hotel found
lavumisa - nearest hotel: White Elephant Safari Lodge
mmathubudukwane - nearest hotel: Makanyane

,Country,Humidity,Lat,City,Lng,Hotel Name
7,US,83,22.0752,kapaa,-159.3190,Pono Kai Resort
10,AR,92,-34.5703,lujan,-59.1050,Bella vita
17,BR,85,-22.9661,arraial do cabo,-42.0278,No hotel found
50,HK,78,22.2855,victoria,114.1577,四季酒店 Four Seasons Hotel
69,US,83,19.7297,hilo,-155.0900,Dolphin Bay Hotel
74,SV,79,13.5928,acajutla,-89.8275,Hotel Lara
106,AU,80,-28.0000,gold coast,153.4333,The Sands Apartments
172,NE,10,19.7500,agadez,10.2500,No hotel found
218,EG,17,24.0934,aswan,32.9070,Yaseen hotel
222,BR,84,-9.0122,maragogi,-35.2225,Pousada Mariluz


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 450,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols=["Country", "Hotel Name"]
)



# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)